In [4]:
# add import of pulp and other things you need here
import pulp as p
import numpy as np

def ex1():
    retval = {}
    retval["x"] = None
    retval["y"] = None
    retval["obj"] = None
    retval["tight_constraints"] = [ None ]
    # Insert your code below:
    
    prob = p.LpProblem("Optimization Problem", p.LpMinimize)

    x = p.LpVariable("x", lowBound=-10)
    y = p.LpVariable("y", upBound=10)

    prob += 122 * x + 143 * y

    prob += p.LpConstraint(e=3 * x + 2 * y, sense=p.LpConstraintLE, rhs=10, name="3x+2y<=10")
    prob += p.LpConstraint(e=12 * x + 14 * y, sense=p.LpConstraintGE, rhs=-12.5, name="12x+14y>=-12.5")
    prob += p.LpConstraint(e=2 * x + 3 * y, sense=p.LpConstraintGE, rhs=3, name="2x+3y>=3")
    prob += p.LpConstraint(e=5 * x - 6 * y,sense= p.LpConstraintGE, rhs=-100, name="5x-6y>=-100")
    
    status = prob.solve()

    retval["x"] = x.value()
    retval["y"] = y.value()
    retval["obj"] = p.value(prob.objective)
    retval["tight_constraints"] = [ i+1 for i in range(4) if (prob.constraints[list(prob.constraints.keys())[i]].value()==0) ]
    
    print("Optimal solution: x = {}, y = {}".format(x.value(), y.value()))
    print("Objective value:", p.value(prob.objective))
    for i in range(4):
        if (prob.constraints[list(prob.constraints.keys())[i]].value())==0:
            print(i+1)
    # Return retval dictionary
    return retval

def ex2():
    retval = {}
    retval['x1'] = None
    retval['x2'] = None
    retval['x3'] = None
    retval['x4'] = None
    retval['x5'] = None
    retval['x6'] = None
    retval['obj'] = None
    # Insert your code below:
    
    payoffs=np.zeros((6,6),dtype=int)

    for i in range(6):
        for k in range(6):
            if ((i-k)==1):
                payoffs[i,k]=-2
            elif((i-k)==-1):
                payoffs[i,k]=2
            elif((i-k)>=2):
                payoffs[i,k]=1
            elif(i==k):
                payoffs[i,k]=0
            else:
                payoffs[i,k]=-1
    prob = p.LpProblem("Mixed Strategy",p.LpMaximize)

    x0 = p.LpVariable("x0")
    x1 = p.LpVariable("x1", lowBound=0, upBound=1)
    x2 = p.LpVariable("x2", lowBound=0, upBound=1)
    x3 = p.LpVariable("x3", lowBound=0, upBound=1)
    x4 = p.LpVariable("x4", lowBound=0, upBound=1)
    x5 = p.LpVariable("x5", lowBound=0, upBound=1)
    x6 = p.LpVariable("x6", lowBound=0, upBound=1)

    prob += x0

    prob += p.LpConstraint(e= x1 + x2 + x3 + x4 + x5 + x6, sense=p.LpConstraintEQ, rhs=1, name="main_constraint")

    x_all = [x1,x2,x3,x4,x5,x6]

    for i in range(6):
        const = p.LpAffineExpression([(x_all[j],payoffs[j][i]) for j in range(6)])
        prob += const >= x0

    solver = p.PULP_CBC_CMD(msg=False)
    result=prob.solve(solver)
    
    retval['x1'] = x_all[0].varValue
    retval['x2'] = x_all[1].varValue
    retval['x3'] = x_all[2].varValue
    retval['x4'] = x_all[3].varValue
    retval['x5'] = x_all[4].varValue
    retval['x6'] = x_all[5].varValue
    retval['obj'] = p.value(prob.objective)

    for i in range(6):
        if i !=5:
            print("{}: {}".format(x_all[i].name,x_all[i].varValue),end=" ")
        else:
            print("{}: {}".format(x_all[i].name,x_all[i].varValue))
    print(p.value(prob.objective))
    # return retval dictionary
    return retval


def ex3():
    retval = {}
    retval['obj'] = None
    retval['x1'] = None
    # there should be retval['xi'] for each company number i
   
    # Insert your code below:

    contracts = np.zeros((69,69),dtype=int)

    with open("hw1-03.txt") as inf:
        for line in inf:
            a,b=line.split()
            contracts[int(a)-1,int(b)-1]+=1
            contracts[int(b)-1,int(a)-1]+=1

    x_all = ["x%d"%i for i in range (1,70,1)]

    x = [p.LpVariable(x_all[i], lowBound = 0, upBound=2, cat="Integer") for i in range(69)]

    prob = p.LpProblem("myStrategy", p.LpMinimize)

    prob += p.lpSum(x)

    for i in range(69):
        for j in range(i+1, 69):
            if contracts[i][j] == 1 and i!=j:
                const = p.LpAffineExpression([(x[i],1),(x[j],1)])
                prob += const >= 2

    solver = p.PULP_CBC_CMD(msg=False)
    result=prob.solve(solver)
    
    retval['obj'] = p.lpSum([x[i].varValue for i in range(69)])
    retval['x1'] = x[0].varValue
    
    for i in range (1,70,1):
        retval["x%d"%i]=x[i-1].varValue

    print(retval["obj"])

    for i in range(69):
        print("representatives from company {}: {:.1f}".format(str(x[i].name)[1:],x[i].varValue))
    print("Total number of representatives involved: {}".format(p.lpSum([x[i].varValue for i in range(69)])))

    # return retval dictionary
    return retval

ex1()
ex2()
ex3()








print(ex1())
a = {}
a.update(ex1())
print(a["x"],a["y"],a["obj"],a["tight_constraints"])

Optimal solution: x = -9.9375, y = 7.625
Objective value: -122.0
2
3
x1: 0.0 x2: 0.0625 x3: 0.3125 x4: 0.25 x5: 0.3125 x6: 0.0625
0.0
61.0
representatives from company 1: 1.0
representatives from company 2: 1.0
representatives from company 3: 1.0
representatives from company 4: 2.0
representatives from company 5: 1.0
representatives from company 6: 1.0
representatives from company 7: 1.0
representatives from company 8: 1.0
representatives from company 9: 1.0
representatives from company 10: 1.0
representatives from company 11: 1.0
representatives from company 12: 1.0
representatives from company 13: 1.0
representatives from company 14: 0.0
representatives from company 15: 1.0
representatives from company 16: 1.0
representatives from company 17: 1.0
representatives from company 18: 1.0
representatives from company 19: 1.0
representatives from company 20: 1.0
representatives from company 21: 1.0
representatives from company 22: 1.0
representatives from company 23: 1.0
representatives fro

In [12]:
def ex3():
    retval = {}
    retval['obj'] = None
    retval['x1'] = None
    # there should be retval['xi'] for each company number i
   
    # Insert your code below:

    contracts = np.zeros((69,69),dtype=int)

    with open("hw1-03.txt") as inf:
        for line in inf:
            a,b=line.split()
            contracts[int(a)-1,int(b)-1]+=1
            contracts[int(b)-1,int(a)-1]+=1

    x_all = ["x%d"%i for i in range (1,70,1)]

    x = [p.LpVariable(x_all[i], lowBound = 0, upBound=2, cat="Integer") for i in range(69)]

    prob = p.LpProblem("myStrategy", p.LpMinimize)

    prob += p.lpSum(x)

    for i in range(69):
        for j in range(i+1, 69):
            if contracts[i][j] == 1 and i!=j:
                const = p.LpAffineExpression([(x[i],1),(x[j],1)])
                prob += const >= 2

    solver = p.PULP_CBC_CMD(msg=False)
    result=prob.solve(solver)
    
    retval['obj'] = p.lpSum([x[i].varValue for i in range(69)])
    retval['x1'] = x[0].varValue
    
    for i in range (1,70,1):
        retval["x%d"%i]=x[i-1].varValue

    # return retval dictionary
    return retval
ex3()


{'obj': 61.0,
 'x1': 1.0,
 'x2': 1.0,
 'x3': 1.0,
 'x4': 2.0,
 'x5': 1.0,
 'x6': 1.0,
 'x7': 1.0,
 'x8': 1.0,
 'x9': 1.0,
 'x10': 1.0,
 'x11': 1.0,
 'x12': 1.0,
 'x13': 1.0,
 'x14': 0.0,
 'x15': 1.0,
 'x16': 1.0,
 'x17': 1.0,
 'x18': 1.0,
 'x19': 1.0,
 'x20': 1.0,
 'x21': 1.0,
 'x22': 1.0,
 'x23': 1.0,
 'x24': 0.0,
 'x25': 1.0,
 'x26': 1.0,
 'x27': 1.0,
 'x28': 1.0,
 'x29': 1.0,
 'x30': 1.0,
 'x31': 1.0,
 'x32': 1.0,
 'x33': 1.0,
 'x34': 0.0,
 'x35': 1.0,
 'x36': 1.0,
 'x37': 1.0,
 'x38': 1.0,
 'x39': 1.0,
 'x40': 1.0,
 'x41': 1.0,
 'x42': 1.0,
 'x43': 1.0,
 'x44': 0.0,
 'x45': 1.0,
 'x46': 1.0,
 'x47': 1.0,
 'x48': 1.0,
 'x49': 1.0,
 'x50': 1.0,
 'x51': 1.0,
 'x52': 1.0,
 'x53': 1.0,
 'x54': 0.0,
 'x55': 1.0,
 'x56': 1.0,
 'x57': 1.0,
 'x58': 1.0,
 'x59': 1.0,
 'x60': 1.0,
 'x61': 1.0,
 'x62': 1.0,
 'x63': 1.0,
 'x64': 0.0,
 'x65': 0.0,
 'x66': 0.0,
 'x67': 0.0,
 'x68': 0.0,
 'x69': 2.0}